In [10]:
!pip install diffusers
!pip install transformers
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git


%cd stylegan2-ada-pytorch


import sys
sys.path.append('.')

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 131 (delta 0), reused 0 (delta 0), pack-reused 129 (from 2)
Receiving objects: 100% (131/131), 1.13 MiB | 28.94 MiB/s, done.
Resolving deltas: 100% (57/57), done.
/content/stylegan2-ada-pytorch


In [11]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "runwayml/stable-diffusion-v1-5"
sd_pipeline = StableDiffusionPipeline.from_pretrained(model_id,
                                                      torch_dtype=torch.float16)
sd_pipeline = sd_pipeline.to("cuda")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
def get_completion_sd(prompt):
  negative_prompt = """
  simple background, duplicate, low quality, lowest quality,
  bad anatomy, bad proportions, extra digits, lowres, username,
  artist name, error, duplicate, watermark, signature, text,
  extra digit, fewer digits, worst quality, jpeg artifacts, blurry
  """
  return sd_pipeline(prompt, negative_prompt=negative_prompt).images[0]

In [ ]:
prompt = """baby llama, wearing red muffler,
grazing, open field, sunset
"""
print(prompt)
sd_image = get_completion_sd(prompt)
sd_image.save("./llama.jpg")

baby llama, wearing red muffler,
grazing, open field, sunset



  0%|          | 0/50 [00:00<?, ?it/s]

### Build AI Image Generator

In [12]:
!pip install gradio==3.48.0

In [13]:
import gradio as gr
from PIL import Image
import numpy as np
from torchvision import transforms


In [14]:
def get_completion(input_type,category, style, color_scheme, pattern, material, fit, occasion, season,custom_prompt):
     if input_type == "Dropdowns":
        prompt = f"A professional fashion model wearing a {category}, designed in {style} style with {color_scheme} colors and a {pattern} pattern. The outfit is crafted from {material}, featuring a {fit} fit, perfect for {occasion} and ideal for {season} wear. The model has a clear, sharp, defined face and a neutral expression, ensuring a fashion-focused presentation. Captured in a high-end fashion studio with professional lighting, this ultra-realistic 8K UHD image highlights the outfit’s intricate textures and design. The model poses confidently against a pure white background, maintaining a clean and professional aesthetic."
     else:
        prompt = custom_prompt


     negative_prompt = """
    simple background, duplicate, low quality, lowest quality,
    bad anatomy, bad proportions, extra digits, lowres, username,
    artist name, error, duplicate, watermark, signature, text,
    extra digit, fewer digits, worst quality, jpeg artifacts, blurry face,
    navel
    """

     return sd_pipeline(prompt, negative_prompt=negative_prompt).images[0]


In [30]:
import dnnlib, legacy
stylegan2_network_pkl = "/content/drive/My Drive/stylegan2-ffhq-config-e.pkl"  # e.g., a local file or URL
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
with dnnlib.util.open_url(stylegan2_network_pkl) as f:
    G = legacy.load_network_pkl(f)['G_ema'].to(device)

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [25]:
model_path = "/content/drive/My Drive/stylegan2-ffhq-config-e.pkl"

In [31]:
def encode_image(image):
    # Dummy encoder: returns a random latent vector of the correct size.
    # Replace this with an actual inversion step.
    return torch.randn([1, G.z_dim]).to(device)

In [32]:
def refine_with_stylegan2(sd_image):
    """
    Refine the Stable Diffusion output using the pretrained StyleGAN2 generator.
    This function inverts the image (dummy inversion here) and then synthesizes a refined image.
    """
    # Ensure image is in RGBA format
    if sd_image.mode != "RGBA":
        sd_image = sd_image.convert("RGBA")
    with torch.no_grad():
        refined_tensor = G.synthesis(latent, noise_mode="const")

    # Post-process: convert tensor from [-1,1] to [0,255]
    refined_tensor = (refined_tensor.clamp(-1, 1) + 1) / 2 * 255
    refined_tensor = refined_tensor[0].permute(1, 2, 0).cpu().numpy().astype(np.uint8)
    return Image.fromarray(refined_tensor)

In [33]:
categories = ["Dress", "T-shirt", "Jeans", "Jacket","Shirt and Jeans","Lehenga","Churidar","Saree","Gown"]
styles = ["Modern", "Vintage", "Casual", "Elegant","Traditional"]
color_schemes = ["Monochrome", "Pastel", "Vibrant"]
patterns = ["Striped", "Floral", "Plain","Embroidery"]
materials = ["Cotton", "Silk", "Denim", "Leather","Georgette","Crepe"]
fits = ["Slim", "Regular", "Loose"]
occasions = ["Formal", "Casual", "Party"]
seasons = ["Summer", "Winter","Normal"]

def show_generator():
    return {welcome_page: gr.update(visible=False), generator_page: gr.update(visible=True)}

# Handle feedback submission
def submit_user_feedback(rating, text):
    return f"Thank you for your feedback! You rated us {rating}/5."

# HTML for start screen
welcome_html = """
<!DOCTYPE html>
<html>
<head>
<style>
  body, html {
    margin: 0;
    padding: 0;
    width: 100%;
    height: 100%;
    font-family: 'Arial', sans-serif;
  }

  .scene {
    width: 100%;
    height: 100vh;
    background: linear-gradient(135deg, #000428, #004e92);
    display: flex;
    justify-content: center;
    align-items: center;
  }

  .card-container {
    width: 60%;
    height: 60%;
    background: black;
    border-radius: 20px;
    display: flex;
    align-items: center;
    justify-content: center;
    box-shadow: 0 10px 30px rgba(0, 0, 0, 0.5);
    text-align: center;
  }

  .title {
    font-size: 3em;
    font-weight: bold;
    color: orange;
  }

  .subtitle {
    font-size: 1.5em;
    color: orange;
    margin-bottom: 20px;
  }

  .start-button {
  padding: 15px 40px;
  background: linear-gradient(135deg, #FFA500, #FF7300); /* Orange gradient */
  color: white; /* Text color */
  font-size: 1.9em;
  border: none;
  border-radius: 50px;
  cursor: pointer;
  box-shadow: 0 5px 15px rgba(255, 165, 0, 0.3);
  transition: transform 0.2s ease-in-out;
}

.start-button:hover {
  transform: scale(1.05);
}

</style>
</head>
<body>
  <div class="scene">
    <div class="card-container">
      <div>
        <h1 class="title">Fashion AI Generator</h1>
        <p class="subtitle">Create stunning fashion designs with AI!</p>
        <br>
        <br>
        <br><br>
        <button class="start-button" onclick="document.getElementById('startButtonGradio').click();">START</button>
      </div>
    </div>
  </div>
</body>
</html>
"""

# Build Gradio UI
with gr.Blocks() as demo:
    # Start Page
    with gr.Group(visible=True) as welcome_page:
        gr.HTML(welcome_html)
        start_btn = gr.Button("Start", visible=False, elem_id="startButtonGradio")

    # Fashion Generator Page (Scrollable)
    with gr.Group(visible=False) as generator_page:
        gr.Markdown("## Fashion Design Generator")

        input_type = gr.Radio(choices=["Dropdowns", "Manual Text"], label="Select Input Type", value="Dropdowns")
        with gr.Row():
            with gr.Column(scale=1):
                category = gr.Dropdown(choices=categories, label="Category", value="Dress")
                style = gr.Dropdown(choices=styles, label="Style", value="Modern")
                color_scheme = gr.Dropdown(choices=color_schemes, label="Color Scheme", value="Vibrant")
                pattern = gr.Dropdown(choices=patterns, label="Pattern", value="Plain")
                material = gr.Dropdown(choices=materials, label="Material", value="Cotton")
                fit = gr.Dropdown(choices=fits, label="Fit", value="Regular")
                occasion = gr.Dropdown(choices=occasions, label="Occasion", value="Casual")
                season = gr.Dropdown(choices=seasons, label="Season", value="Summer")

                custom_prompt = gr.Textbox(label="Custom Description", placeholder="Describe your fashion design...", interactive=True)
                generate_btn = gr.Button("Generate Fashion Image", variant="primary")

            with gr.Column(scale=1):
                output_image = gr.Image(label="Generated Fashion Design", type="pil")

        # Feedback Section (Scroll Enabled)
        with gr.Group():
            gr.Markdown("### We'd Love Your Feedback!")
            rating = gr.Slider(minimum=1, maximum=5, step=1, label="Rate (1-5)", value=5)
            feedback_text = gr.Textbox(label="Your Feedback", lines=3, placeholder="Your thoughts on the generated design")
            submit_feedback = gr.Button("Submit Feedback")
            feedback_message = gr.Markdown(visible=True)

    # Event Handlers
    start_btn.click(show_generator, outputs=[welcome_page, generator_page])
    generate_btn.click(get_completion, inputs=[input_type,category, style, color_scheme, pattern, material, fit, occasion, season,custom_prompt], outputs=output_image)
    submit_feedback.click(submit_user_feedback, inputs=[rating, feedback_text], outputs=feedback_message)


# Launch Gradio App
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
IMPORTANT: You are using gradio version 3.48.0, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://d2f18a291f6ec3bcc8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
